In [ ]:
from google.colab import drive
from nltk.corpus import stopwords
from nltk.corpus import gutenberg
from collections import defaultdict
import nltk
from nltk.stem.porter import PorterStemmer
import heapq

porter = PorterStemmer()

In [ ]:
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


True

In [ ]:
import re
def remove_symbols(line):
  return re.sub('[^A-Za-z0-9/s]+', '', line).lower()

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv, sys

csv.field_size_limit(sys.maxsize)

9223372036854775807

In [ ]:
% cd /content/drive/MyDrive/gutenberg/

/content/drive/MyDrive/gutenberg


In [ ]:
!ls

gutenberg_data.csv


In [ ]:
BLOCK_SIZE = 100000

In [ ]:
def bsbi():
  freq_dict = defaultdict(set)
  with open('gutenberg_data.csv') as f:
    next(f)
    csv_file = csv.reader(f)
    total_files = 0
    i = 0
    current_block = 0

    for line in csv_file:
      title, author, link, id, bookshelf, text = line
      i = i + 1

      # if (i == 2):
      #     break

      for word in text.split():
        word = remove_symbols(word)

        if word and word not in stop_words:
          word = porter.stem(word)

          if word not in freq_dict:
            current_block = current_block + 1

          if not freq_dict[word].__contains__(id):
            freq_dict[word].add(id)
            current_block = current_block + 1

        

        if current_block >= BLOCK_SIZE:
          sorted_list = sorted(freq_dict.items(), key= lambda _: _[0])

          with open(f'./IR2_OUTPUTS/OP{total_files}.txt', 'w') as f:
          # with open(f'./IRO/OP{total_files}.txt', 'w') as f:
            for word_id, doc_ids in sorted_list:
              f.write(word_id)
              for doc_id in doc_ids:
                f.write(f' {doc_id}')
              f.write('\n')
            
          current_block = 0
          freq_dict.clear()
          total_files = total_files + 1
            
          print (i, ' rows done!')

    sorted_list = sorted(freq_dict.items(), key= lambda _: _[0])
    
    print (len(sorted_list))
    if len(sorted_list) > 0:
      with open(f'./IR2_OUTPUTS/OP{total_files}.txt', 'w') as f:
      # with open(f'./IRO/OP{total_files}.txt', 'w') as f:
        for word_id, doc_ids in sorted_list:
          
          f.write(word_id)
          for doc_id in doc_ids:
            
            f.write(f' {doc_id}')
          f.write('\n')
          

      current_block = 0
      freq_dict.clear()
      total_files = total_files + 1

    print (i, ' rows done')

bsbi()

15  rows done!
23  rows done!
41  rows done!
54  rows done!
67  rows done!
81  rows done!
96  rows done!
116  rows done!
130  rows done!
146  rows done!
157  rows done!
180  rows done!
207  rows done!
220  rows done!
254  rows done!
271  rows done!
280  rows done!
291  rows done!
302  rows done!
332  rows done!
353  rows done!
371  rows done!
387  rows done!
400  rows done!
444  rows done!
456  rows done!
468  rows done!
476  rows done!
488  rows done!
499  rows done!
506  rows done!
509  rows done!
518  rows done!
526  rows done!
533  rows done!
541  rows done!
548  rows done!
564  rows done!
590  rows done!
606  rows done!
616  rows done!
627  rows done!
634  rows done!
641  rows done!
649  rows done!
660  rows done!
677  rows done!
685  rows done!
692  rows done!
702  rows done!
717  rows done!
730  rows done!
740  rows done!
748  rows done!
757  rows done!
762  rows done!
772  rows done!
779  rows done!
788  rows done!
796  rows done!
804  rows done!
816  rows done!
828  rows done!

In [ ]:
!ls

gutenberg_data.csv  IR2_OUTPUTS


In [ ]:
file_names = [f'./IR2_OUTPUTS/OP{i}.txt' for i in range(947)]
file_pointers = [open(i) for i in file_names]

In [ ]:
"""
This algo is called external merge
In python I took help of heaps to do the merging
A great answer of what I did is at this link,
https://stackoverflow.com/questions/1001569/python-class-to-merge-sorted-files-how-can-this-be-improved
^Make sure to take a look at this to see what we are really doing

"""

# here we are using yeild so we are JUST READING ONE LINE at a time
# you can read more about it by searching generators in python
def decorated_file(f, key):
  for line in f:
    yield (key(line), line)

files = map(open, file_names)
outfile = open('./IROUTPUTS_v2/merged.txt', 'w')

def key_fn(line):
    return line.split(' ', 2)[0] # returning word_id

"""
The simple sorting version would be like below

for line in heapq.merge(*[decorated_file(f, keyfunc) for f in files]):
    outfile.write(line[1])

so if there'd be 2 files 
a 1 2 3
b 3 4 5

and 

a 6 7 8
b 8 9 10

o/p would be
a 1 2 3
a 6 7 8
b 3 4 5
b 8 9 10
So I kind of did lil bit of modification to do what we want in one pass
TIME: 6m 14s (for ~1k files of size ~620kb each)

a 1 2 3 6 7 8
b 3 4 5 8 9 10
"""
prev = ''
for line in heapq.merge(*[decorated_file(f, key_fn) for f in files]):
  # to understand this, you can do this over small number of sorted files and try to print line
  if prev != line[0]:
    # if we have new word, make sure to add new line at first
    outfile.write(f'\n{line[1].strip()}')
    prev = line[0]
  # if we have same word yet, put a space and add other ids
  else:
    # line[1][len(line[0]):] => We are removing the word_id string and then writing the line
    outfile.write(f' {line[1][len(line[0]):].strip()}')
for i in file_pointers:
  i.close()
outfile.close()